In [1]:
import os
import warnings

import polars as pl
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
from scipy.special import erfc
from lets_plot import (
    LetsPlot,
    element_blank,
    theme,
    geom_point,
    geom_line,
    scale_x_log10,
    scale_y_log10,
    labs,
    scale_fill_brewer,
    scale_color_brewer,
    gggrid,
    ggplot,
    ggsize,
    lims,
    theme_classic, aes,
    scale_color_viridis,
    scale_fill_viridis,
    element_text,
    element_geom,
    scale_color_manual,
    scale_fill_manual,
    scale_fill_hue,
    scale_color_hue,
    geom_histogram,
    theme_void,
    geom_density,
)
LetsPlot.setup_html()

In [2]:
def double_exp(x, a, b, c, d):
    return a * np.exp(-x * b) + (d) * np.exp(-x * c)


def erfc_exp(t, a, koff, ks):
    return (a 
            * np.exp(( np.square(koff) * t) / ks) 
            * erfc(np.sqrt((np.square(koff) * t) / ks))
    )


def powerlaw(x, a, b):
    return a * x ** (-b)


def exp_decay(x, a, b):
    return a * np.exp(-x * b)


def tri_exp(x, a, b, c, d, e, f):
    return a * np.exp(-x * b) + c * np.exp(-x * d) + e * np.exp(-x * f)


def quad_exp(x, a, b, c, d, e, f, g, h):
    return (
        a * np.exp(-x * b)
        + c * np.exp(-x * d)
        + e * np.exp(-x * f)
        + g * np.exp(-x * h)
    )


def penta_exp(x, a, b, c, d, e, f, g, h, i, j):
    return (
        a * np.exp(-x * b)
        + c * np.exp(-x * d)
        + e * np.exp(-x * f)
        + g * np.exp(-x * h)
        + i * np.exp(-x * j)
    )

def deleteNaN(y: np.ndarray,t: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """
    delete NaN parts of the input array and time array opened for it,
    and returns time array and values array.

    """

    t = t[~np.isnan(y)]
    y = y[~np.isnan(y)]

    return t, y

def value_fit(
    val: np.ndarray, t: np.ndarray, tmax: int, 
    eq: callable,
    delete_nan: bool = True
) -> tuple[np.ndarray, np.ndarray, tuple]:
    """

    Parameters
    ----------
    val : np.ndarray
        Values 1d array to fit.
    eq : callable
        Equation to create a fit.

    Returns
    -------
    y_fit : np.ndarray
        1d Fitted values array.
    ss_res_norm : np.ndarray
        Sum of squares of residuals normalized.
    popt : tuple

    """
    t_range = np.arange(tmax) + 1

    if delete_nan:
        t, val = deleteNaN(val,t)

    popt, _ = curve_fit(eq, t, val, maxfev=20000000)


    y_fit = eq(t_range, *popt)  # full time length
    # y_fit[y_fit < 1] = np.nan  # too small values to be removed
    # y_fit[y_fit > np.max(val) * 2] = np.nan  # too big values removed

    return y_fit



In [3]:

def arr_minimize(arr: np.ndarray, method: str = "median") -> np.ndarray:
    """
    Minimizes 1d array by removing repeats, according to the given method.

    Parameters
    ----------
    arr : np.ndarray
        1d array to be minimized.
    method : str, optional
        'median' or 'average'. The default is 'median'.

    Returns
    -------
    arr1 : np.ndarray
        minimized array.

    """

    search = np.unique(arr)  # arr of unique elements
    search = search[search > 0]  # remove nans

    arr1 = arr.copy()

    for s in search:
        (positions,) = np.where(arr == s)
        if method == "median":
            mid = int(np.median(positions))

        elif method == "average":
            mid = int(np.average(positions))
        elif method == "max":
            mid = int(np.max(positions))
        elif method == "min":
            mid = int(np.min(positions))

        arr1[positions] = np.nan
        arr1[mid] = s  # mid value is kept

    return arr1


def df_minimize(df: pd.DataFrame, **kwargs) -> pd.DataFrame:
    """

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame to be minimized.

    Returns
    -------
    df : pd.DataFrame
        Minimized DataFrame.

    """
    for i in range(len(df.columns)):
        df.iloc[:, i] = arr_minimize(
            df.iloc[:, i], **kwargs
        )  # values minimized and returned

    return df

In [4]:
# values dataframe
df = pd.read_csv("./data/duration_cont.csv", index_col=None)
df = df_minimize(df, method="median")
df[df == 0] = np.nan
df = df.dropna(axis=0, how="all")
df["timestep"] = df.index + 1
df = pl.from_pandas(df)

In [5]:
df.tail()

uniform_250,uniform_350,gaus_250,gaus_350,3.50_60,timestep
f64,f64,f64,f64,f64,i64
null,2.0,null,null,null,1499
null,null,null,2.0,null,1605
null,null,null,null,1.0,1772
null,1.0,null,null,null,2038
null,null,null,1.0,null,2080


In [6]:
def sample_ends_favored_1D(df: pd.DataFrame, n: int = 10) -> pd.DataFrame:
    """sample a dataframe but favor the ends

    Parameters
    ----------
    df : pd.DataFrame

    n : int, optional
        end size, by default 10

    Returns
    -------
    pd.DataFrame
        sampled dataframe
    """


    mid_df = df.iloc[n:-n]
    sampled_df = pd.concat(
        [df.head(n), mid_df.sample(frac=0.20, random_state=42), df.tail(n)],
        ignore_index=True
    )
    return sampled_df

In [7]:
def sample_EF_1D(df: pl.DataFrame, n: int = 10) -> pd.DataFrame:
    """sample a dataframe but favor the ends

    Parameters
    ----------
    df : pd.DataFrame

    n : int, optional
        end size, by default 10

    Returns
    -------
    pd.DataFrame
        sampled dataframe
    """

    

    mid_df = df.slice(n, len(df) - 2 * n)
    sampled_df = pl.concat(
        [df.head(n), mid_df.sample(fraction=0.20, seed=42), df.tail(n)],
        how="vertical",
    ).unpivot(index="timestep", 
              value_name="remaining", 
              variable_name="case")
    
    return sampled_df

In [8]:
def sample_EF_2D(df: pl.DataFrame, n: int = 10) -> pd.DataFrame:

    all_parts = [
        sample_EF_1D(df.select([col, "timestep"]), n=n).with_columns(
            pl.lit(col).alias("case")
        )
        for col in df.columns if col != "timestep"
    ]
    collected = pl.concat(all_parts, how="vertical")

    return collected.drop_nulls()

In [9]:
def add_type_um(df: pl.DataFrame) -> pl.DataFrame:

    return df.with_columns(
        pl.col("case").str.split("_").list.get(0).alias("type"),
        pl.col("case").str.split("_").list.get(1).alias("um"),
    )


In [10]:
df = sample_EF_2D(df, n=40)
df

timestep,case,remaining
i64,str,f64
1,"""uniform_250""",1.192739e6
2,"""uniform_250""",779689.0
3,"""uniform_250""",541090.0
4,"""uniform_250""",389455.0
5,"""uniform_250""",288740.0
…,…,…
996,"""3.50_60""",6.0
1061,"""3.50_60""",5.0
1124,"""3.50_60""",4.0


In [11]:
df = df.with_columns(
    pl.col("case").str.split("_").list.get(0).alias("distribution"),
    pl.col("case").str.split("_").list.get(1).alias("around"),
)

In [12]:
df350 = df.filter(pl.col("case") == "3.50_60")

In [13]:
df.filter(pl.col("case") == "uniform_350").sort("timestep").tail()

timestep,case,remaining,distribution,around
i64,str,f64,str,str
962,"""uniform_350""",6.0,"""uniform""","""350"""
1026,"""uniform_350""",5.0,"""uniform""","""350"""
1153,"""uniform_350""",4.0,"""uniform""","""350"""
1499,"""uniform_350""",2.0,"""uniform""","""350"""
2038,"""uniform_350""",1.0,"""uniform""","""350"""


In [14]:
df.filter(pl.col("case") == "gaus_350").sort("timestep").tail()

timestep,case,remaining,distribution,around
i64,str,f64,str,str
1168,"""gaus_350""",5.0,"""gaus""","""350"""
1289,"""gaus_350""",4.0,"""gaus""","""350"""
1388,"""gaus_350""",3.0,"""gaus""","""350"""
1605,"""gaus_350""",2.0,"""gaus""","""350"""
2080,"""gaus_350""",1.0,"""gaus""","""350"""


In [15]:
cases = df.select("case").unique().to_series().to_list()
cases

['3.50_60', 'gaus_250', 'uniform_250', 'gaus_350', 'uniform_350']

In [16]:
df.select("timestep").to_series().max()

2080

In [17]:
# make fit
tmax = df.select("timestep").to_series().max()
fits_arr = list()
fits = pl.DataFrame().with_columns(
    pl.int_range(1, tmax + 1).alias("timestep"),
)
cases = df.select("case").unique().to_series().to_list()
for c in cases:
    if not c == "3.50_60":
        case_df = df.filter(pl.col("case") == c)
        timestep = case_df.select("timestep").to_numpy()
        data = case_df.select("remaining").to_numpy()
    
        fits = fits.with_columns(
            pl.Series(value_fit(data, timestep, tmax=tmax, eq=tri_exp)).alias(f"{c}_tri.exp"),
            pl.Series(value_fit(data, timestep, tmax=tmax, eq=quad_exp)).alias(f"{c}_quad.exp"),
            pl.Series(value_fit(data, timestep, tmax=tmax, eq=powerlaw)).alias(f"{c}_powerlaw"),
        )

C:\Users\zafi_\AppData\Local\Temp\ipykernel_11756\2893836686.py:21: RuntimeWarning: overflow encountered in exp
  return a * np.exp(-x * b) + c * np.exp(-x * d) + e * np.exp(-x * f)
C:\Users\zafi_\AppData\Local\Temp\ipykernel_11756\2893836686.py:26: RuntimeWarning: overflow encountered in exp
  a * np.exp(-x * b)
C:\Users\zafi_\AppData\Local\Temp\ipykernel_11756\2893836686.py:27: RuntimeWarning: overflow encountered in exp
  + c * np.exp(-x * d)
C:\Users\zafi_\AppData\Local\Temp\ipykernel_11756\2893836686.py:28: RuntimeWarning: overflow encountered in exp
  + e * np.exp(-x * f)
C:\Users\zafi_\AppData\Local\Temp\ipykernel_11756\2893836686.py:29: RuntimeWarning: overflow encountered in exp
  + g * np.exp(-x * h)
C:\Users\zafi_\AppData\Local\Temp\ipykernel_11756\2893836686.py:13: RuntimeWarning: overflow encountered in power
  return a * x ** (-b)
C:\Users\zafi_\AppData\Local\Temp\ipykernel_11756\2893836686.py:13: RuntimeWarning: overflow encountered in multiply
  return a * x ** (-b)


In [18]:
# make fit
tmax = df.select("timestep").to_series().max()
fits_arr = list()
fits = pl.DataFrame().with_columns(
    pl.int_range(1, tmax + 1).alias("timestep"),
)
cases = df.select("case").unique().to_series().to_list()
for c in cases:
    if not c == "3.50_60":
        case_df = df.filter(pl.col("case") == c)
        timestep = case_df.select("timestep").to_numpy()
        data = case_df.select("remaining").to_numpy()
    
        fits = fits.with_columns(
            pl.Series(value_fit(data, timestep, tmax=tmax, eq=tri_exp)).alias(f"{c}_tri.exp"),
            pl.Series(value_fit(data, timestep, tmax=tmax, eq=erfc_exp)).alias(f"{c}_erfc.exp"),
        )

C:\Users\zafi_\AppData\Local\Temp\ipykernel_11756\2893836686.py:21: RuntimeWarning: overflow encountered in exp
  return a * np.exp(-x * b) + c * np.exp(-x * d) + e * np.exp(-x * f)
C:\Users\zafi_\AppData\Local\Temp\ipykernel_11756\2893836686.py:7: RuntimeWarning: overflow encountered in exp
  * np.exp(( np.square(koff) * t) / ks)
C:\Users\zafi_\AppData\Local\Temp\ipykernel_11756\2893836686.py:6: RuntimeWarning: invalid value encountered in multiply
  return (a
C:\Users\zafi_\AppData\Local\Temp\ipykernel_11756\2893836686.py:82: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(eq, t, val, maxfev=20000000)
C:\Users\zafi_\AppData\Local\Temp\ipykernel_11756\2893836686.py:8: RuntimeWarning: invalid value encountered in sqrt
  * erfc(np.sqrt((np.square(koff) * t) / ks))
C:\Users\zafi_\AppData\Local\Temp\ipykernel_11756\2893836686.py:6: RuntimeWarning: overflow encountered in multiply
  return (a


In [19]:
fits

timestep,gaus_350_tri.exp,gaus_350_erfc.exp,uniform_350_tri.exp,uniform_350_erfc.exp,gaus_250_tri.exp,gaus_250_erfc.exp,uniform_250_tri.exp,uniform_250_erfc.exp
i64,f64,f64,f64,f64,f64,f64,f64,f64
1,25514.630246,0.427584,11832.233498,0.427584,2.6448e6,1.1320e6,1.1926e6,556080.75035
2,22247.21202,0.336204,10714.146251,0.336204,1.4870e6,810456.685144,780436.707191,407502.00153
3,19574.471789,0.287341,9779.927878,0.287341,871834.760306,664602.625452,539969.50762,337145.769898
4,17371.847913,0.255396,8993.945529,0.255396,524096.472221,576828.104686,389330.151503,293999.196935
5,15542.309972,0.232326,8327.776569,0.232326,320514.583742,516616.829801,289323.808374,264080.372057
…,…,…,…,…,…,…,…,…
2076,4.1691e-13,NaN,3.5518e-9,NaN,0.0,NaN,5.5419e-97,NaN
2077,4.0959e-13,NaN,3.5050e-9,NaN,0.0,NaN,4.9521e-97,NaN
2078,4.0239e-13,NaN,3.4588e-9,NaN,0.0,NaN,4.4251e-97,NaN


In [20]:
fitsm = fits.unpivot(index="timestep",
                     value_name="remaining",
                     variable_name="fit_case")

In [21]:
fitsm

timestep,fit_case,remaining
i64,str,f64
1,"""gaus_350_tri.exp""",25514.630246
2,"""gaus_350_tri.exp""",22247.21202
3,"""gaus_350_tri.exp""",19574.471789
4,"""gaus_350_tri.exp""",17371.847913
5,"""gaus_350_tri.exp""",15542.309972
…,…,…
2076,"""uniform_250_erfc.exp""",NaN
2077,"""uniform_250_erfc.exp""",NaN
2078,"""uniform_250_erfc.exp""",NaN


In [22]:
fitsm = fitsm.with_columns(
    pl.col("fit_case").str.split("_").list.get(0).alias("distribution"),
    pl.col("fit_case").str.split("_").list.get(1).alias("around"),
    pl.col("fit_case").str.split("_").list.get(2).alias("equation")
)

In [23]:
gauss = fitsm.filter(pl.col("distribution")=="gaus",pl.col("remaining").is_not_nan())
uni = fitsm.filter(pl.col("distribution")=="uniform",pl.col("remaining").is_not_nan())

In [24]:
gauss.sample(12)

timestep,fit_case,remaining,distribution,around,equation
i64,str,f64,str,str,str
902,"""gaus_250_tri.exp""",8.3039e-149,"""gaus""","""250""","""tri.exp"""
1664,"""gaus_350_tri.exp""",6.2039e-10,"""gaus""","""350""","""tri.exp"""
646,"""gaus_350_tri.exp""",0.042836,"""gaus""","""350""","""tri.exp"""
780,"""gaus_350_tri.exp""",0.00398,"""gaus""","""350""","""tri.exp"""
646,"""gaus_350_erfc.exp""",0.022181,"""gaus""","""350""","""erfc.exp"""
…,…,…,…,…,…
1682,"""gaus_250_tri.exp""",3.6780e-282,"""gaus""","""250""","""tri.exp"""
146,"""gaus_250_tri.exp""",1.4783e-19,"""gaus""","""250""","""tri.exp"""
1103,"""gaus_350_tri.exp""",0.000013,"""gaus""","""350""","""tri.exp"""


In [25]:
df.sample(12)

timestep,case,remaining,distribution,around
i64,str,f64,str,str
41,"""uniform_250""",1772.0,"""uniform""","""250"""
7,"""gaus_350""",12725.0,"""gaus""","""350"""
321,"""gaus_350""",80.0,"""gaus""","""350"""
1379,"""3.50_60""",2.0,"""3.50""","""60"""
19,"""uniform_250""",20017.0,"""uniform""","""250"""
…,…,…,…,…
962,"""uniform_350""",6.0,"""uniform""","""350"""
58,"""uniform_350""",1736.0,"""uniform""","""350"""
558,"""gaus_350""",25.0,"""gaus""","""350"""


In [26]:
df_gauss =df.filter(pl.col("distribution")=="gaus")

In [27]:
eqs = ["tri.exp","erfc.exp"]

In [28]:
def plot_it(df:pl.DataFrame,fits:pl.DataFrame,color_by):

    return (ggplot()
            + geom_point(data=df, mapping=aes(x="timestep", y="remaining",color=color_by),fill="#dfdfdf",shape=21, stroke=1, size=5)
            + geom_line(data=fits, mapping=aes(x="timestep", y="remaining",color=color_by),size=3,alpha=1)
            + geom_line(data=df350,mapping=aes(x="timestep", y="remaining"),size=2,alpha=0.7, color="black")
            + scale_color_brewer(labels=["1-4kT", "2-5kT"])
            + scale_fill_brewer(labels=["1-4kT", "2-5kT"])
            + scale_x_log10(format="~e")
            + scale_y_log10(format=".0~e")
            + theme_classic()
            + ggsize(800,400)
            + lims(x=(0.72, 3.7e3),y=(0.5, 1.1e7))
            + theme(legend_title=element_blank(), 
                    exponent_format="pow",
                    axis_text=element_text(size=20,color="#1f1f1f"),
                    axis_title=element_text(size=21,color="#1f1f1f"),
                    legend_text=element_text(size=20),
                    legend_position=[0.8, 0.8]
                    )
            + labs(x="Duration (a.u.)", y="Occurence (n)")
            
            )

In [29]:
grid_gauss =[
        plot_it(df=df.filter(pl.col("distribution")=="gaus").sort("around"),
                fits=gauss.filter(pl.col("equation")==eq).sort("around"),
                color_by="around") 
                for eq in eqs 
                ]

In [30]:
grid_uni= [
        plot_it(df=df.filter(pl.col("distribution")=="uniform").sort("around"),
                fits=uni.filter(pl.col("equation")==eq).sort("around"),
                color_by="around")+ scale_fill_hue(labels=["1-4kT", "2-5kT"])+scale_color_hue(labels=["1-4kT", "2-5kT"])
                for eq in eqs 
                ]

In [31]:
all = gggrid(grid_gauss+grid_uni,ncol=2)+ggsize(900,700)
all

In [32]:
# all.to_svg('../Figures/fig6__ERFC.svg')